In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
import pandas as pd

train_data = pd.read_csv('datasets/images/sign_mnist_train.csv')
test_data = pd.read_csv('datasets/images/sign_mnist_test.csv')

In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27455 entries, 0 to 27454
Columns: 785 entries, label to pixel784
dtypes: int64(785)
memory usage: 164.4 MB


In [4]:
train_data

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27450,13,189,189,190,190,192,193,193,193,193,...,132,165,99,77,52,200,234,200,222,225
27451,23,151,154,157,158,160,161,163,164,166,...,198,198,198,198,198,196,195,195,195,194
27452,18,174,174,174,174,174,175,175,174,173,...,121,196,209,208,206,204,203,202,200,200
27453,17,177,181,184,185,187,189,190,191,191,...,119,56,27,58,102,79,47,64,87,93


In [5]:
from tensorflow import one_hot
train_labels = train_data.iloc[:, 0]
train_images = train_data.iloc[:, 1:] / 255

test_labels = test_data.iloc[:, 0]
test_images = test_data.iloc[:, 1:] / 255

train_labels_onehot = one_hot(train_labels, 25)
test_labels_onehot = one_hot(test_labels, 25)

In [6]:
train_labels_onehot

<tf.Tensor: shape=(27455, 25), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)>

#  Keras Model

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import Precision, Recall 
model = Sequential()

model.add(Dense(784, activation='relu'))
model.add(Dense(392, activation='relu'))
model.add(Dense(196, activation='relu'))
model.add(Dense(25, activation='softmax'))

In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [13]:
model.fit(train_images.to_numpy(), train_labels_onehot.numpy(), batch_size=64, epochs=50)

Train on 27455 samples
Epoch 1/50
27455/27455 [==============================] - 1s 37us/sample - loss: 0.1071
Epoch 2/50
27455/27455 [==============================] - 1s 36us/sample - loss: 0.1575
Epoch 3/50
27455/27455 [==============================] - 1s 36us/sample - loss: 0.0040
Epoch 4/50
27455/27455 [==============================] - 1s 37us/sample - loss: 9.5278e-04
Epoch 5/50
27455/27455 [==============================] - 1s 36us/sample - loss: 6.6119e-04
Epoch 6/50
27455/27455 [==============================] - 1s 36us/sample - loss: 4.8485e-04
Epoch 7/50
27455/27455 [==============================] - 1s 36us/sample - loss: 3.9886e-04
Epoch 8/50
27455/27455 [==============================] - 1s 36us/sample - loss: 0.2364
Epoch 9/50
27455/27455 [==============================] - 1s 36us/sample - loss: 0.0474
Epoch 10/50
27455/27455 [==============================] - 1s 38us/sample - loss: 0.0058
Epoch 11/50
27455/27455 [==============================] - 1s 35us/sample - loss

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  615440    
_________________________________________________________________
dense_1 (Dense)              multiple                  307720    
_________________________________________________________________
dense_2 (Dense)              multiple                  77028     
_________________________________________________________________
dense_3 (Dense)              multiple                  4925      
Total params: 1,005,113
Trainable params: 1,005,113
Non-trainable params: 0
_________________________________________________________________


In [15]:
import numpy as np

predict_test_labels = np.argmax(model.predict(test_images), axis=-1)
print(predict_test_labels)

[ 6  5 10 ...  2  4  2]


In [16]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score
#confusion_matrix(test_labels, predict_test_labels)
print('Precision: {}'.format(precision_score(test_labels, predict_test_labels, average='macro')))
print('Recall: {}'.format(recall_score(test_labels, predict_test_labels, average='macro')))

Precision: 0.8052231848280722
Recall: 0.8055522322481847
